# Настройка ноутбука

In [2]:
import pandas as pd
import numpy as np
from sklearn import datasets
import datetime as dt

from pandasql import sqldf

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# Расширить рабочее поле ноутбука на весь экран
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Описание

Ноутбук содержит описание оператора оконных функций в языке SQL, а также практики его использования.

# Теория

В SQL есть несколько типов функций. Скалярные функции принимают на вход одно значение и возвращают тоже одно (например concat, upper, lower). Агрегирующие функции принимают на вход набор значений и возращают одно значение (sum, min, max, avg и т.д.). 

Есть также третий тип функций, которые принимают на вход набор строк, выполняют преобразования в контексте этого набора и возвращают приобразованные значения  - это оконные функции (RANK, DENSE_RANK, ROW_NUMBER). 

<img src="../../img/window_functions.PNG" width="800">

### Синтаксис

```sql
<название функции>(<выражение>) OVER (
    <окно>
  <сортировка>
  <границы окна>
)
```

```sql
select 
    id, date
    , avg(time) over(partition by id order by date rows between 1 preceding and current row) 
    as avg_time
FROM data
```

```sql
ROWS BETWEEN 1 PRECEDING AND CURRENT ROW
ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
ROWS BETWEEN 15 PRECEDING AND 15 FOLLOWING
```

### Виды оконных функций

Оконные функции делятся на:
- Агрегатные функции
- Ранжирующие функции
- Функции смещения
- Аналитические функции

---
**Агрегатные функции - Aggregate functions**

Собственно, те же, что и обычные, только встроенные в конструкцию с OVER: 

SUM, AVG, COUNT, MIN, MAX

---
**Ранжируемые функции - Numbering functions**

ROW_NUMBER() — нумерует строки в результирующем наборе.

RANK() — присваивает ранг для каждой строки, если найдутся одинаковые значения, то следующий ранг присваивается с пропуском.

DENSE_RANK() — присваивает ранг для каждой строки, если найдутся одинаковые значения, то следующий ранг присваивается без пропуска.

PERCENT_RANK() - присваивает персентиль значений.


NTILE() — помогает разделить результирующий набор на группы.

---
**Функции смещения / навигации -  Navigation functions**

Используется в случаях, когда нужно обратиться к строке в наборе данных из окна с некоторым смещением относительно текущей строки.

LAG() — смещение назад

LEAD() — смещение вперед

LAG() и LEAD() имеют следующие аргументы:
    - Столбец, значение которого необходимо вернуть
    - На сколько строк выполнить смешение (дефолт =1)
    - Что вставить, если вернулся NULL
    

FIRST_VALUE() — найти первое значение набора данных

LAST_VALUE() — найти последнее значение набора данных

# Практики

#### Примеры задач

- [Stratascratch: Marketing Campaign Success [Advanced] - Hard](https://platform.stratascratch.com/coding/514-marketing-campaign-success-advanced?tabname=question)

### Пример

In [5]:
exams = pd.DataFrame({
    'name': [
        'Robert Pattison', 'Robert Paulson', 'Rex Roberts', 'Thomas Edison', 'Bonny Bones', 'Elison Woods',
        'Robert Pattison', 'Robert Paulson', 'Rex Roberts', 'Thomas Edison', 'Bonny Bones', 'Elison Woods',
    ],
    'exam_id': [
        1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2,
    ],
    'grade': [
        5, 2, 3, 4, 3, 5,
        4, 4, 4, 5, 4, 5,
    ],
})

exams.head()

,name,exam_id,grade
0,Robert Pattison,1,5
1,Robert Paulson,1,2
2,Rex Roberts,1,3
3,Thomas Edison,1,4
4,Bonny Bones,1,3


In [6]:
query = """
select 
      row_number() over () as row_number
    , name
    , grade
FROM 
    exams
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,row_number,name,grade
0,1,Robert Pattison,5
1,2,Robert Paulson,2
2,3,Rex Roberts,3
3,4,Thomas Edison,4
4,5,Bonny Bones,3
5,6,Elison Woods,5
6,7,Robert Pattison,4
7,8,Robert Paulson,4
8,9,Rex Roberts,4
9,10,Thomas Edison,5


In [7]:
query = """
select
      row_number() over (partition by name) as row_number
    , name
    , grade
FROM 
    exams
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,row_number,name,grade
0,1,Bonny Bones,3
1,2,Bonny Bones,4
2,1,Elison Woods,5
3,2,Elison Woods,5
4,1,Rex Roberts,3
5,2,Rex Roberts,4
6,1,Robert Pattison,5
7,2,Robert Pattison,4
8,1,Robert Paulson,2
9,2,Robert Paulson,4


## Посчитать процентное изменение

```sql 
select 
      to_char(created_at::date, 'YYYY-MM') as year_month
    , round(
        (sum(value) - lag(sum(value),1) over (w)) *100/lag(sum(value),1) over (w), 2
    ) as revenue_diff
from df
group by year_month
window w as (order by to_char(created_at::date, 'YYYY-MM'))
```

#### Прмеры задач

- [Stratascratch: Monthly Percentage Difference - Hard](https://platform.stratascratch.com/coding/10319-monthly-percentage-difference?tabname=question)

## Посчитать отклонение от среднего

```sql
select 
    val - avg(val) over (partition by month)
from data
```

#### Примеры задач:

- [Stratascratch: Distance Per Dollar - Hard](https://platform.stratascratch.com/coding/10302-distance-per-dollar/solutions?code_type=1)

## Отранжировать значения 

ROW_NUMBER() — нумерует строки в результирующем наборе.

RANK() — присваивает ранг для каждой строки, если найдутся одинаковые значения, то следующий ранг присваивается с пропуском.

DENSE_RANK() — присваивает ранг для каждой строки, если найдутся одинаковые значения, то следующий ранг присваивается без пропуска.

PERCENT_RANK() - присваивает персентиль значений.

...В каждой группе (от максимального к минимальному)

```sql 
select
        gr, val
      , row_number() over w
      , rank() over w
      , dense_rank() over w
      , percent_rank() over w
from df
window w as (partition by gr order by val desc)
```

#### Пример

In [47]:
data = pd.DataFrame({'val': [
    0,
    1, 1, 1, 1, 1,
    2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
    3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
    4, 4, 4,
    5, 5, 5, 5, 5,
    6, 6, 6
]})

In [48]:
query = """
select
      val
    , row_number() over w as row_number
    , rank() over w as rank
    , dense_rank() over w as dense_rank
    , round(cume_dist() over w, 2) as cum_dist
    , round(percent_rank() over w, 2) as pct_rank
from data
window w as (order by val desc)
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,val,row_number,rank,dense_rank,cum_dist,pct_rank
0,6,1,1,1,0.08,0.00
1,6,2,1,1,0.08,0.00
2,6,3,1,1,0.08,0.00
3,5,4,4,2,0.22,0.08
4,5,5,4,2,0.22,0.08
5,5,6,4,2,0.22,0.08
6,5,7,4,2,0.22,0.08
7,5,8,4,2,0.22,0.08
8,4,9,9,3,0.30,0.22
9,4,10,9,3,0.30,0.22


### Вывести топ20% наибольших значений

```sql
```

In [58]:
query = """
with 
ranged_tab as (
    select
          val
        , rank() over (order by val desc) as rank
        , count(*) over () as n_val
        , round((rank() over (order by val desc)) / cast(count(*) over () as real), 4) as pct_rank
        , round(percent_rank() over (order by val desc), 4) as pct_rank_func
    from data
)
select * from ranged_tab where pct_rank <= 0.2
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,val,rank,n_val,pct_rank,pct_rank_func
0,6,1,37,0.0270,0.0000
1,6,1,37,0.0270,0.0000
2,6,1,37,0.0270,0.0000
3,5,4,37,0.1081,0.0833
4,5,4,37,0.1081,0.0833
5,5,4,37,0.1081,0.0833
6,5,4,37,0.1081,0.0833
7,5,4,37,0.1081,0.0833


### Отранжировать агрегаты

```sql 
select 
      id_col
    , sum(value_col) as values_sum
    , dense_rank() over (order by sum(value_col) desc)
from df
group by id_col
```

#### Примеры задач:

- [Stratascratch: Ranking Most Active Guests - Medium](https://platform.stratascratch.com/coding/10159-ranking-most-active-guests?tabname=question)
- [Stratascratch: Activity Rank - Medium](https://platform.stratascratch.com/coding/10351-activity-rank/solutions?code_type=1)
- [Stratascratch: Find the top 5 cities with the most 5 star businesses - Medium](https://platform.stratascratch.com/coding/10148-find-the-top-10-cities-with-the-most-5-star-businesses?code_type=1)
- [Stratascratch: Most Checkins - Medium](https://platform.stratascratch.com/coding/10053-most-checkins/solutions?code_type=1)

### Процентный ранг значения (в контексте групп)

```sql
select 
      percent_rank() over (partition by group_name order by val asc)
--     , 1 - ntile(100) over (partition by state order by fraud_score desc)/100::decimal as percentile
from data
```

#### Примеры задач:

- [Stratascratch: Top Percentile Fraud - Medium](https://platform.stratascratch.com/coding/10303-top-percentile-fraud?code_type=1)
- [Stratascratch: Ranking Hosts By Beds - Medium](https://platform.stratascratch.com/coding/10161-ranking-hosts-by-beds/solutions?code_type=1)
- [Stratascratch: Start Dates Of Top Drivers - Medium](https://platform.stratascratch.com/coding/10083-start-dates-of-top-drivers?code_type=1)

## Выделить первое, последнее и n-ное значение

Вывести первое, последнее и n-ное значение в группе 

```sql
select 
      distinct gr
    , first_value(val) over w as first
    , last_value(val) over w as last 
    , nth_value(val, {n}) over w as nth
from data
window w as (partition by gr)
```

#### Альтернативные решения

##### Выделить n-ную строку

```sql
select 
    gr, val
from (
    select
          gr 
        , row_number() over (partition by gr) as row_number
        , val
    FROM data
)
where row_number = {n}
```

##### Выделить последнюю строку

```sql
with ordered_data as (
    select *, row_number() over () as order_column from data
)

select 
    gr, val
from (
    select *, row_number() over (partition by group_col order by order_column desc) as row_number
    from ordered_data
) subquery
where 
    row_number = 1
```

#### Пример

In [23]:
data = pd.DataFrame({
    'gr': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'C'],
    'val': ['a1','a2', 'a3', 'b1', 'b2', 'c1', 'c2', 'c3'],
})

data.head()

,gr,val
0,A,a1
1,A,a2
2,A,a3
3,B,b1
4,B,b2


In [24]:
n = 2

query = f"""

select 
      distinct gr
    , first_value(val) over w as first
    , last_value(val) over w as last 
    , nth_value(val, {n}) over w as nth
from data
window w as (partition by gr)
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,gr,first,last,nth
0,A,a1,a3,a2
1,B,b1,b2,b2
2,C,c1,c3,c2


In [8]:
n = 3

query = f"""

select 
    gr, val
from (
    select
          gr 
        , row_number() over (partition by gr) as row_number
        , val
    FROM data
)
where row_number = {n}

"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,gr,val
0,A,a3
1,C,c3


In [21]:
query = '''

with ordered_data as (
    select *, row_number() over () as order_column from data
)

select 
    gr, val
from (
    select *, row_number() over (partition by gr order by order_column desc) as row_number
    from ordered_data
) subquery
where 
    row_number = 1
'''

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,gr,val
0,A,a3
1,B,b2
2,C,c3


## Вывести строки с минимальным и максимальным значением

```sql
-- Через cte + union
with 

min_max_vals as (
    (select gr, max(val) as val, 'Highest val' as val_type from data group by gr)
    union all
    (select gr, min(val) as val, 'Lowest val' as val_type from data group by gr)
)

select
    d.worker_id, d.salary, d.department, mm.salary_type
from data d right join min_max_vals mm on d.gr = mm.gr and d.val = mm.val
```

```sql
-- Через cte + window row_number


ranked_vals as (
    select *
        , row_number() over (partition by gr order by val desc) as max_rank
        , row_number() over (partition by gr order by val asc) as min_rank
        , (
        case
            when row_number() over (partition by gr order by val desc) = 1 then 'Highest val'
            when row_number() over (partition by gr order by val asc) = 1 then 'Lowest val' 
        end
        ) as val_type
    from data

)

select 
    data_id, val, gr, val_type
from ranked_vals
where max_rank = 1 or min_rank = 1
```

```sql
-- Через cte + window max

with item_cnt_tab as (
    select
          month
        , sum(unit_price*quantity) as revenue
        , max(sum(unit_price*quantity)) over (partition by month)
    from data 
    group by month, item_name
)
select month, item_name, revenue from item_cnt_tab where revenue = max_revenue

```

#### Примеры задач:

- [Stratascratch: Workers With The Highest And Lowest Salaries - Medium](https://platform.stratascratch.com/coding/10152-workers-with-the-highest-and-lowest-salaries/solutions?code_type=1)
- [Stratascratch: Best Selling Item - Hard](https://platform.stratascratch.com/coding/10172-best-selling-item/solutions?code_type=1)

## Работа с последовательностями строк

### Вывести предыдущую, текущую и следующую строки в контексте группы

```sql
select  
      group_col
    , lag(created_at, 1) over (partition by group_col order by created_at asc) as prev_row
    , created_at as cur_row
    , lead(created_at, 1) over (partition by group_col order by created_at asc) as next_row
from data
```

##### Примеры задач:

- [Stratascratch: Finding User Purchases - Medium](https://platform.stratascratch.com/coding/10322-finding-user-purchases?code_type=1)

##### Пример:

In [32]:
transactions = pd.DataFrame({
    'id': [
        1, 2, 3, 4, 
        5, 6, 
        7, 
        8, 9, 10, 11, 12, 
        13, 14, 15
    ],
    'user_id': [
        1, 1, 1, 1, 
        2, 2, 
        3, 
        4, 4, 4, 4, 4, 
        5, 5, 5
    ],
    'created_at': [
        dt.datetime(2023, 4, 21, 14, 0, 0), dt.datetime(2023, 4, 21, 14, 5, 30), dt.datetime(2023, 4, 21, 14, 10, 40), dt.datetime(2023, 4, 21, 17, 10, 40), 
        dt.datetime(2023, 4, 21, 15, 30, 0), dt.datetime(2023, 4, 21, 15, 40, 0), 
        dt.datetime(2023, 4, 22, 13, 25, 25),
        dt.datetime(2023, 4, 21, 16, 0, 0), dt.datetime(2023, 4, 22, 16, 0, 0), dt.datetime(2023, 4, 24, 16, 0, 0), dt.datetime(2023, 4, 26, 16, 0, 0), dt.datetime(2023, 4, 27, 16, 0, 0),
        dt.datetime(2023, 4, 21, 13, 30, 35), dt.datetime(2023, 4, 21, 13, 36, 35), dt.datetime(2023, 4, 21, 15, 20, 38),
    ],
    'amount': [
        5, 6, 10, 2, 
        15, 8, 
        16, 
        13, 2, 3, 9 , 4, 
        4, 9, 3
    ],
})

transactions

,id,user_id,created_at,amount
0,1,1,2023-04-21 14:00:00,5
1,2,1,2023-04-21 14:05:30,6
2,3,1,2023-04-21 14:10:40,10
3,4,1,2023-04-21 17:10:40,2
4,5,2,2023-04-21 15:30:00,15
5,6,2,2023-04-21 15:40:00,8
6,7,3,2023-04-22 13:25:25,16
7,8,4,2023-04-21 16:00:00,13
8,9,4,2023-04-22 16:00:00,2
9,10,4,2023-04-24 16:00:00,3


In [40]:
query = '''

select  
      user_id
    , lag(created_at, 1) over (partition by user_id order by created_at asc) as prev_transaction
    , created_at as cur_transaction
    , lead(created_at, 1) over (partition by user_id order by created_at asc) as next_transaction
from transactions
'''

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,user_id,prev_transaction,cur_transaction,next_transaction
0,1,None,2023-04-21 14:00:00.000000,2023-04-21 14:05:30.000000
1,1,2023-04-21 14:00:00.000000,2023-04-21 14:05:30.000000,2023-04-21 14:10:40.000000
2,1,2023-04-21 14:05:30.000000,2023-04-21 14:10:40.000000,2023-04-21 17:10:40.000000
3,1,2023-04-21 14:10:40.000000,2023-04-21 17:10:40.000000,None
4,2,None,2023-04-21 15:30:00.000000,2023-04-21 15:40:00.000000
5,2,2023-04-21 15:30:00.000000,2023-04-21 15:40:00.000000,None
6,3,None,2023-04-22 13:25:25.000000,None
7,4,None,2023-04-21 16:00:00.000000,2023-04-22 16:00:00.000000
8,4,2023-04-21 16:00:00.000000,2023-04-22 16:00:00.000000,2023-04-24 16:00:00.000000
9,4,2023-04-22 16:00:00.000000,2023-04-24 16:00:00.000000,2023-04-26 16:00:00.000000


### Посчитать скользящее среднее

```sql
select 
    month, avg(val) over (order by month rows 2 preceding)
from data
```

#### Примеры задач:

- [Stratascratch: Revenue Over Time - Hard](https://platform.stratascratch.com/coding/10314-revenue-over-time/solutions?code_type=1)